In [1]:
import math, operator

import jupyter_cadquery as jcq
from jupyter_cadquery.replay import show

from build123d import *

Overwriting auto display for cadquery Workplane and Shape


In [2]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
    height=400,
)

In [3]:
%%html
<style>
   /* Fix widget background color in VS Code */
   .cell-output-ipywidget-background {
      background-color: transparent !important;
   }
</style>

In [4]:
case_size = [76, 200, 200]
case_foot_r = 12 / 2 + 0.7
case_foot_h = 5 + 0.5
case_foot_left_pos = 13 + 4
case_foot_right_pos = 16
case_foot_top_pos = 12 + 3
case_foot_bottom_pos = 12 + 3
case_foots = [
    (x, y)
    for x in [case_foot_left_pos, case_foot_right_pos]
    for y in [case_foot_top_pos, case_foot_bottom_pos]
]
foot_offsets = [
    (x, y)
    for x in [-case_size[1] / 2, case_size[2] / 2]
    for y in [-case_size[1] / 2, case_size[2] / 2]
]
case_foots = map(
    lambda a, b: tuple(
        map(lambda p, offset: offset - p if offset > 0 else offset + p, a, b)
    ),
    case_foots,
    foot_offsets,
)
x_offset_of_power_port = 27
y_offset_of_power_port = 16.5
power_port_r = 12 / 2
power_port_screws_spacing = 18
power_port_screw_head_r = 5.5 / 2 + 0.5
power_port_screw_head_h = 3+0.2

with BuildPart() as the_case:
    Box(76, 200, 200)
    with BuildSketch(
        Plane(the_case.faces().sort_by(Axis.X)[-1].center(), (0, 1, 0), (1, 0, 0))
    ):
        with Locations(*(case_foots)):
            Circle(case_foot_r)
    Extrude(amount=case_foot_h)
    with BuildPart(
        Plane(
            (
                case_size[0] / 2,
                case_size[1] / 2,
                -case_size[2] / 2,
            ),
            (-1, 0, 0),
            (0, 1, 0),
        )
    ):
        with Locations((x_offset_of_power_port, y_offset_of_power_port, 0)):
            Cylinder(power_port_r, 40)
            with BuildSketch():
                with Locations(
                    (power_port_screws_spacing / 2, 0, 0),
                    (-power_port_screws_spacing / 2, 0, 0),
                ):
                    Circle(power_port_screw_head_r)
                MakeHull()
            Extrude(amount=power_port_screw_head_h)


show(the_case)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=400, pinning=False, theme='browser', title=None…

In [5]:
class StandBlock(BasePartObject):
    def __init__(
        self,
        size: float,
        pad_r: float,
        pad_depth: float,
        rotation: RotationLike = (0, 0, 0),
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildPart() as b:
            Box(size, size, size)
            with BuildSketch(b.faces().sort_by(Axis.Z)[0]):
                Circle(pad_r)
            Extrude(amount=-pad_depth, mode=Mode.SUBTRACT)
        super().__init__(solid=b.part, rotation=rotation, align=align, mode=mode)


with BuildPart():
    show(StandBlock(10, 5, 1))

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=400, pinning=False, theme='browser', title=None…

In [6]:
stand_block_size = 40
block_offset = 10
block_thickness = stand_block_size/2-block_offset
foot_pad_r = 15
foot_pad_depth = 1

In [7]:

with BuildPart() as stand_f:
    with GridLocations(
        x_spacing=case_size[0] - block_offset * 2, y_spacing=0, x_count=2, y_count=1
    ):
        StandBlock(stand_block_size, foot_pad_r, foot_pad_depth)
    with BuildSketch() as sk:
        Rectangle(stand_block_size/2, block_thickness, align=(Align.CENTER, Align.MAX))
    with Locations((0, stand_block_size/2, -10)):
        # https://github.com/gumyr/build123d/issues/142
        Extrude(*sk.faces(), amount=-1)
    with Locations(
        (0, case_size[1] / 2 - block_offset, case_size[2] / 2 - block_offset)
    ):
        Add(the_case.part, mode=Mode.SUBTRACT)
 
show(stand_f.part)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=400, pinning=False, theme='browser', title=None…

In [8]:
with BuildPart() as stand_b:
    with GridLocations(
        x_spacing=case_size[0] - block_offset * 2, y_spacing=0, x_count=2, y_count=1
    ):
        StandBlock(stand_block_size, foot_pad_r, foot_pad_depth)
    with BuildSketch() as sk:
        Rectangle(stand_block_size/2, block_thickness, align=(Align.CENTER, Align.MAX))
    with Locations((0, stand_block_size/2, -10)):
        # https://github.com/gumyr/build123d/issues/142
        r = Extrude(*sk.faces(), amount=-1, mode=Mode.PRIVATE)
        Mirror(r, about=Plane.XZ)
    with Locations(
        (0, -case_size[1] / 2 + block_offset, case_size[2] / 2 - block_offset)
    ):
        Add(the_case.part, mode=Mode.SUBTRACT)

show(stand_b.part)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=400, pinning=False, theme='browser', title=None…

In [10]:
stand_f.part.export_step("/tmp/f.step")
stand_b.part.export_step("/tmp/b.step")
the_case.part.export_step("/tmp/x.step")


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/f.step(1538 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/b.step(2002 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)                 ******

******

<IFSelect_ReturnStatus.IFSelect_RetDone: 1>

 Step File Name : /tmp/x.step(1424 ents)  Write  Done
